In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time
from selenium import webdriver

from selenium.webdriver.chrome.options import Options  # For options like headless browser
from selenium.webdriver.chrome.service import Service   # For automatic ChromeDriver installation
from webdriver_manager.chrome import ChromeDriverManager # For automatic ChromeDriver installation
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By # For element selection on page
import random
from urllib.parse import urlparse, parse_qs
options = Options()
#options.add_argument('--headless')  # Disable first so we can observe what's happening
options.add_argument('--incognito')
driver_service = Service('D:\\Statista\\chromedriver-win32\\chromedriver.exe')
driver = webdriver.Chrome(service=driver_service, options=options)

In [26]:
def scrape(URL, category):
    driver.get(URL)
    time.sleep(3)

    #accept cookies
    try:
        driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
    except:
        print("No cookie button!")
    
    output_dict = {
        "productID": [],
        "colorID": [],
        "name": [], 
        "brand": [], 
        "category": [],
        "price": [],
        "originalPrice": [],
        "currency": [],
        "url": [],
        "imgURL":[],
        "country": [],
        "date": [] 
    }

    products = driver.find_elements(By.CLASS_NAME, "suchen-produktliste__item.suchen-produktliste__item--produkt")

    for prod in products:
        num_of_color = int(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__farbkacheln-container").get_dom_attribute("data-anzahl-alle-artikel"))

        if num_of_color == 1: #just one color
            output_dict['productID'].append(prod.find_element(By.CSS_SELECTOR, "div.suchen-produkt.suchen-produktliste__produkt.suchen-produkt--imageratio > a").get_attribute("data-syan"))
            output_dict['colorID'].append(prod.find_element(By.CSS_SELECTOR, "div.suchen-produkt.suchen-produktliste__produkt.suchen-produkt--imageratio > a").get_attribute("data-farb-id"))
            output_dict['name'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__detail > span.suchen-produkt__name").text)
            output_dict['brand'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__detail > span.suchen-produkt__marke").text)
            output_dict['category'].append(category)
            #price 
            try: #not on sale
                output_dict['price'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-preis.suchen-preis--schwarzpreis").text.replace(" €", ""))
                output_dict['originalPrice'].append(pd.NA)
            except: # on sale
                output_dict['price'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__preis__content > ins").text.replace(" €", ""))
                output_dict['originalPrice'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__preis__content > del").text.replace(" €", ""))
            
            output_dict['currency'].append("EUR")
            output_dict['url'].append(prod.find_element(By.CSS_SELECTOR, "div.suchen-produkt.suchen-produktliste__produkt.suchen-produkt--imageratio > a").get_attribute("href"))
            output_dict['imgURL'].append(prod.find_element(By.CSS_SELECTOR, "div.suchen-produkt.suchen-produktliste__produkt.suchen-produkt--imageratio > a > span.suchen-produkt__bild-container > picture > img").get_attribute("src"))
            output_dict['country'].append("DEU")
            output_dict['date'].append(datetime.now().strftime("%Y-%m-%d"))
        
        else: #more than 1 color
            for i in range(1, num_of_color+1):
                output_dict['productID'].append(prod.find_element(By.CSS_SELECTOR, "div.suchen-produkt.suchen-produktliste__produkt.suchen-produkt--imageratio > a").get_attribute("data-syan"))           
                selector = f"span.suchen-produkt__farbkacheln-container > span.suchen-produkt__farbkacheln > a:nth-child({i})"
                output_dict['colorID'].append(prod.find_element(By.CSS_SELECTOR, selector).get_attribute("data-farb-id"))
                output_dict['name'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__detail > span.suchen-produkt__name").text)
                output_dict['brand'].append(prod.find_element(By.CSS_SELECTOR, "span.suchen-produkt__detail > span.suchen-produkt__marke").text)
                output_dict['category'].append(category)
                output_dict['currency'].append("EUR")
                output_dict['country'].append("DEU")
                output_dict['date'].append(datetime.now().strftime("%Y-%m-%d"))

                product_url = prod.find_element(By.CSS_SELECTOR, selector).get_attribute("href")
                output_dict['url'].append(product_url)
                output_dict["imgURL"].append(prod.find_element(By.CSS_SELECTOR, selector).get_attribute("data-image-standard"))

                # Open a new tab using JavaScript
                driver.execute_script("window.open('', '_blank');")
                # Switch to the new tab
                driver.switch_to.window(driver.window_handles[1])

                # Navigate to the new URL in the new tab
                driver.get(product_url)
                time.sleep(1)

                try:
                    output_dict['price'].append(driver.find_element(By.CSS_SELECTOR, "div.bewerten-preisbereich > span").text.replace(" €", ""))
                    output_dict['originalPrice'].append(pd.NA)
                except:
                    output_dict['price'].append(driver.find_element(By.CSS_SELECTOR, "div.bewerten-preisbereich > ins").text.replace(" €", ""))
                    output_dict['originalPrice'].append(driver.find_element(By.CSS_SELECTOR, "div.bewerten-preisbereich > del").text.replace(" €", ""))
                # Close the new tab
                driver.close()
                # Switch back to the original tab
                driver.switch_to.window(driver.window_handles[0])
    output_df = pd.DataFrame(output_dict)
    return output_df



In [25]:
df = scrape(URL="https://www.breuninger.com/de/damen/bekleidung/hosen/", category="Hose")
df

No cookie button!


,productID,colorID,name,brand,category,price,originalPrice,currency,url,imgURL,country,date
0,1001442411,4131a6b7226b423c8519b6b2a9856c8b,Sweatpants,POLO RALPH LAUREN,Hose,"149,99",<NA>,EUR,https://www.breuninger.com/de/marken/polo-ralp...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
1,1001442411,4eac9a7ee0cf4059805a631d34146927,Sweatpants,POLO RALPH LAUREN,Hose,"149,99",<NA>,EUR,https://www.breuninger.com/de/marken/polo-ralp...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
2,1001442411,85d903e349114e45aeb0aeb2511fa039,Sweatpants,POLO RALPH LAUREN,Hose,"99,99","149,99",EUR,https://www.breuninger.com/de/marken/polo-ralp...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
3,1001442411,31ab6e6c5b2a46d7a28048be6bd0a791,Sweatpants,POLO RALPH LAUREN,Hose,"79,99","149,99",EUR,https://www.breuninger.com/de/marken/polo-ralp...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
4,1001442411,6f1c5fc1ada2421e9cbaa81d741d33be,Sweatpants,POLO RALPH LAUREN,Hose,"149,99",<NA>,EUR,https://www.breuninger.com/de/marken/polo-ralp...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
...,...,...,...,...,...,...,...,...,...,...,...,...
217,1001487239,b008385d912844a6b4e8a6f01e818571,Leggings in Lederoptik,MRS & HUGS,Hose,"159,99",<NA>,EUR,https://www.breuninger.com/de/marken/mrs-hugs/...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
218,1001487239,76944b8e03a1414bad4d164c3af66951,Leggings in Lederoptik,MRS & HUGS,Hose,"119,99","159,99",EUR,https://www.breuninger.com/de/marken/mrs-hugs/...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
219,1001487239,1998bf93f7f14a30a1497a1631c262db,Leggings in Lederoptik,MRS & HUGS,Hose,"119,99","159,99",EUR,https://www.breuninger.com/de/marken/mrs-hugs/...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10
220,1001487239,36261328bb38480cb1070442e72985df,Leggings in Lederoptik,MRS & HUGS,Hose,"159,99",<NA>,EUR,https://www.breuninger.com/de/marken/mrs-hugs/...,https://cms.brnstc.de/product_images/287x393/c...,DEU,2023-12-10


In [ ]:
def multi_scrape(URL, category):
    complete_df = scrape(URL, category)
    
    try:
        next_page = driver.find_element(By.CSS_SELECTOR, "span.suchen-pager__next").get_attribute('data-href')
        next_url = "https://www.breuninger.com/" + next_page
    except:
        "no next page!"